In [1]:
from nltk.corpus import words

word_list = words.words()

print(len(word_list))

236736


In [2]:
print(word_list[:10])

['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron']


In [3]:
letters = "abcdefghijklmnopqrstuvwxyz"
letters = [letter for letter in letters]

def get_next_letter(word):
    last_letter = word[len(word)-1].lower()
    #print(word, last_letter)
    last_index = letters.index(last_letter)
    next_letter = letters[(last_index + 1)%len(letters)]
    #print(next_letter)
    return next_letter
    

In [4]:
import random
data_size = 10000
random.seed(13)
random.shuffle(word_list)

word_list = word_list[:data_size]

In [5]:
print(len(word_list))

10000


In [13]:
x = []
y = []

for word in word_list:
    x.append(word.lower())
    y.append(get_next_letter(word))
    
y = [ord(char) for char in y]
print(x[:10], y[:10])


['oysterbird', 'uncinula', 'heterotropia', 'tahkhana', 'doohickey', 'knickers', 'asymptotical', 'supersignificant', 'unwallet', 'saintdom'] [101, 98, 98, 98, 122, 116, 109, 117, 117, 110]


In [14]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

sorted_words = sorted(word_list, key=len)
#print(sorted_words[-1])
max_word_len = len(sorted_words[-1])

#max word len = 23
print(max_word_len)
word_seqs = [[ord(char) for char in word] for word in word_list]

input_sequences = np.array(pad_sequences(word_seqs, maxlen=max_word_len))
#word = [ord(char) for char in word]   
print(input_sequences[:10])
print(y[:10])

23
[[  0   0   0   0   0   0   0   0   0   0   0   0   0 111 121 115 116 101
  114  98 105 114 100]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  85 110  99
  105 110 117 108  97]
 [  0   0   0   0   0   0   0   0   0   0   0 104 101 116 101 114 111 116
  114 111 112 105  97]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 116  97 104
  107 104  97 110  97]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 100 111 111 104
  105  99 107 101 121]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 107 110 105
   99 107 101 114 115]
 [  0   0   0   0   0   0   0   0   0   0   0  97 115 121 109 112 116 111
  116 105  99  97 108]
 [  0   0   0   0   0   0   0 115 117 112 101 114 115 105 103 110 105 102
  105  99  97 110 116]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 117 110 119
   97 108 108 101 116]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 115  97 105
  110 116 100 111 109]]
[101, 98, 98, 98, 122, 116

In [23]:
import keras.utils as ku
print(input_sequences.shape)
#print(ord('z'))
#122 is the ord of z, which should be the last possible char
labels = ku.to_categorical(y, num_classes=123)

print(labels[0])

(10000, 23)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [29]:
def get_char(prediction):
    char_index = np.where(prediction == 1)
    #first index accesses array of found locations, second accesses first value
    return chr(char_index[0][0])
    
print(get_char(labels[0]))

e


In [31]:
index = int(.9 * data_size)
train_x = input_sequences[:index]
test_x = input_sequences[index:]
train_y = labels[:index]
test_y = labels[index:]

print(train_x[0])
print(get_char(train_y[0]))

[  0   0   0   0   0   0   0   0   0   0   0   0   0 111 121 115 116 101
 114  98 105 114 100]
e


In [36]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

input_len = max_word_len
num_chars=123

model = Sequential()

#number of distinct words in training set, size of embedding vectors, 
#size of each input sequence
#second param was 10 before
model.add(Embedding(num_chars, 32, input_length = input_len))

model.add(LSTM(100))

model.add(Dropout(0.1))

#softmax squashes the values between 0 and 1 but makes sure they add to 1
#the values are proprtional to the number of values
model.add(Dense(num_chars, activation='softmax'))

#calculates with an ouput between 0 and 1
model.compile(loss='categorical_crossentropy', optimizer = 'adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 23, 32)            3936      
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 123)               12423     
Total params: 69,559
Trainable params: 69,559
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(train_x, train_y, epochs=10, verbose=1)

Epoch 1/10
9000/9000 [==============================] - 5s 508us/step - loss: 1.2702e-07
Epoch 2/10
9000/9000 [==============================] - 5s 532us/step - loss: 1.3047e-07
Epoch 3/10
9000/9000 [==============================] - 6s 654us/step - loss: 1.2987e-07
Epoch 4/10
9000/9000 [==============================] - 5s 548us/step - loss: 1.3234e-07 0s - 
Epoch 5/10
9000/9000 [==============================] - 5s 523us/step - loss: 1.2895e-07
Epoch 6/10
9000/9000 [==============================] - 5s 509us/step - loss: 1.2909e-07
Epoch 7/10
9000/9000 [==============================] - 4s 494us/step - loss: 1.2794e-07
Epoch 8/10
9000/9000 [==============================] - 5s 533us/step - loss: 1.3161e-07
Epoch 9/10
9000/9000 [==============================] - 5s 547us/step - loss: 1.2837e-07
Epoch 10/10
9000/9000 [==============================] - 5s 520us/step - loss: 1.2601e-07


In [52]:
model.save("shacks.model")

In [45]:
test_word = test_x[0]
print(test_word.shape)

predicted = model.predict(test_x)

(23,)


In [49]:

for i in range(10):
    print(predicted[i])
    print(get_char(predicted[i]))
    print(get_char(test_y[i]))

[3.82134848e-14 6.98636705e-14 9.74203221e-14 1.38683146e-13
 6.18221566e-14 6.69910701e-14 1.90630254e-13 9.74636225e-14
 3.01681253e-14 2.00857777e-13 8.14059513e-14 8.91326401e-14
 7.51006042e-14 2.25704600e-14 1.18078195e-13 5.56472931e-14
 6.31731741e-14 7.01649432e-14 6.29847601e-14 2.43000708e-14
 5.40059263e-14 1.04345840e-13 1.08725623e-13 2.97046188e-13
 6.93624438e-14 5.42305391e-14 2.08622419e-14 8.89522830e-14
 1.14807361e-13 1.14210019e-13 2.33967685e-13 6.10811654e-14
 2.20609203e-13 8.92798205e-14 8.72121521e-14 1.10198566e-13
 1.54867533e-13 2.39883200e-13 1.74377494e-13 7.34824528e-14
 3.95339685e-14 6.21919847e-14 5.77164388e-14 5.21073595e-14
 2.73473210e-14 1.66836122e-13 2.37145915e-13 5.36687055e-14
 4.52260909e-14 1.17507295e-13 1.59626679e-13 4.17091708e-13
 6.54366020e-14 1.02055510e-13 3.31893882e-13 5.14101938e-14
 5.52714578e-14 6.58378449e-14 1.00545941e-13 3.50681364e-14
 6.13230373e-14 5.22672183e-14 1.34624923e-13 9.98699143e-14
 5.41381447e-14 2.083329

In [56]:
def make_guess():
    words = input("Please enter a word or multiple words separated by spaces: ")
    word_list = words.split(" ")
    #print(word_list)
    words_data = [[ord(char) for char in word] for word in word_list]
    model_input = np.array(pad_sequences(words_data, maxlen=max_word_len))
    predictions = model.predict(model_input)
    for i in range(len(predictions)):
        print("Based on your input", word_list[i], ", the next character alphabetically is:", get_char(predictions[i]))
    
#make_guess()

Please enter a word or multiple words separated by spaces: apple bottom jeans jeans
Based on your input apple , the next character alphabetically is: f
Based on your input bottom , the next character alphabetically is: n
Based on your input jeans , the next character alphabetically is: t
Based on your input jeans , the next character alphabetically is: t
